In [15]:
! pip install -r ./requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
from langchain.document_loaders import PDFPlumberLoader, DirectoryLoader

loader = DirectoryLoader("./DEMO/", glob="**/*.pdf", loader_cls=PDFPlumberLoader)

docs = loader.load()

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
)

chunks = splitter.split_documents(docs)

In [18]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large",
    model_kwargs={"device": device},
)

# embeddings.embed_query(chunks[0].page_content)

from langchain_community.vectorstores.chroma import Chroma
from chromadb.config import Settings

setting = Settings(
    anonymized_telemetry=False,
    is_persistent=True,
)

vector_store = Chroma(
    persist_directory='.DB_llama3',
    embedding_function=embeddings,
    client_settings=setting,
)
retriever = vector_store.as_retriever()

load INSTRUCTOR_Transformer


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/models/Dense.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no lon

max_seq_length  512


In [19]:
vn_template = """
<s> [INST] Bạn là một trợ lý hữu ích. Hãy trả lời câu hỏi sau dựa trên ngữ cảnh. [/INST] </s>
[INST] Câu hỏi: {question} Ngữ cảnh: {context} Trả lời: [/INST]
"""

vn_5_question_template = """
<s> [INST] Bạn là một trợ lý hữu ích. Hãy tạo ra 5 câu hỏi tương tự với câu hỏi đã cho. Chỉ liệt kê câu các câu hỏi. Chỉ sử dụng ngữ cảnh được cung cấp để tạo ra các câu hỏi, không sử dụng bất kỳ thông tin nào ngoài ngữ cảnh này. Nếu bạn không biết, hãy nói rằng bạn không biết. [/INST] </s>
[INST]
Câu hỏi: {question}
Trả lời: [/INST]
"""

vn_question = "Tại sao có quy định cấm hành khách đi chân trần khi bay?"

In [20]:
from langchain.prompts import PromptTemplate
from difflib import SequenceMatcher

# Step 1: Define the prompt template to generate similar questions
# template = PromptTemplate.from_template(
#     """
#     <s> [INST] You are a helpful assistant. Generate 5 questions that are similar to the given question. Use only the provided context for generating the questions, do not use any information outside of this context. If you don't know just say that you don't know.[/INST] </s>
#     [INST] 
#     Question: {question} 
#     Answer: [/INST]
#     """
# )


template = PromptTemplate.from_template(
    vn_5_question_template
)

In [21]:
# Step 2: Initialize the Llama model
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="llama3", base_url='http://127.0.0.1:11434')

In [22]:
# Step 3: Define the original question and context
question = vn_question

In [23]:
# Step 4: Format the prompt to generate similar questions
formatted_prompt = template.format(question=question)

In [24]:
# Step 5: Generate 5 similar questions using the model
response = llm.invoke([("human", formatted_prompt)])
content = response.content
generated_questions = content.split("\n")  # Assuming the model returns the 5 questions line by line


In [25]:
response

AIMessage(content='Here are 5 similar questions:\n\n1. Tại sao các sân bay phải có quy định về giới hạn chất lỏng mang vào máy bay?\n2. tại sao các hãng hàng không yêu cầu khách hành mang hộ chiếu và giấy tờ tùy thân khi đăng ký chuyến bay?\n3. tại sao việc check-in online là bắt buộc đối với tất cả các chuyến bay?\n4. Tại sao an toàn mạng là vấn đề quan trọng nhất trong việc thiết lập đường truyền internet trên máy bay?\n5. tại sao có quy định về giới hạn số lượng đồ vật mang vào cabin trên các chuyến bay?', response_metadata={'model': 'llama3', 'created_at': '2024-09-28T12:20:45.147344643Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 10699103719, 'load_duration': 7230317328, 'prompt_eval_count': 122, 'prompt_eval_duration': 181938000, 'eval_count': 122, 'eval_duration': 3225793000}, id='run-ebbd0d06-82ed-45d4-b25e-e8a929fae8d5-0')

In [26]:
context = retriever

In [27]:
# Step 6: Send each generated question to the model and store responses
responses = []
for q in generated_questions:
    question_prompt = vn_template
    formatted_question_prompt = vn_template.format(question=q,context=context)
    answer = llm.invoke([("human", formatted_question_prompt)])
    responses.append(answer)

In [28]:
def compare_answers(ans1, ans2, threshold=0.8):
    # Use a simple similarity ratio to compare two answers
    similarity_ratio = SequenceMatcher(None, ans1, ans2).ratio()
    return similarity_ratio > threshold  # Return True if similar, False otherwise
  
# Function to check if all answers are meaningfully equivalent
def are_answers_meaningfully_equivalent(responses):
    for i in range(len(responses)):
        for j in range(i + 1, len(responses)):
            if not compare_answers(responses[i], responses[j]):
                return False  # Return False if any pair is not similar
    return True  # Return True if all pairs are similar

In [29]:
# Check if all responses are meaningfully equivalent
are_equivalent = are_answers_meaningfully_equivalent(responses)

# Print the result
print(f"All answers are meaningfully equivalent: {are_equivalent}")

TypeError: unhashable type: 'dict'